In [3]:
import numpy as np
import h5py
import matplotlib.pyplot as plt
from dnn_utils import sigmoid, sigmoid_backward, relu, relu_backward

import copy
%matplotlib inline
plt.rcParams['figure.figsize'] = (5.0, 4.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

%load_ext autoreload
%autoreload 2

np.random.seed(1)

# Step 1 - initialise_parameters

In [13]:
def initialize_parameters(n_x, n_h, n_y):
    np.random.seed(1)
    
    W1 = np.random.randn(n_h,n_x) * 0.01
    b1 = np.zeros((n_h,1))
    W2 = np.random.randn(n_y,n_h) * 0.01
    b2 = np.zeros((n_y,1))
    
    parameters = {
        "W1" : W1,
        "b1" : b1,
        "W2" : W2,
        "b2" : b2,
    }
    
    return parameters

In [14]:
n_x, n_h, n_y = (3,2,1)

parameters = initialize_parameters(n_x, n_h, n_y)

for key in parameters:
    print(key + ": ",parameters[key])

W1:  [[ 0.01624345 -0.00611756 -0.00528172]
 [-0.01072969  0.00865408 -0.02301539]]
b1:  [[0.]
 [0.]]
W2:  [[ 0.01744812 -0.00761207]]
b2:  [[0.]]


# Step 2: Initialise Parameters for an L-layer Neural Network

In [17]:
def initialize_parameters_deep(layer_dims):
    
    np.random.seed(3)
    parameters = {}
    L = len(layer_dims) # number of layers in the network
    
    for l in range(1, L):
        
        parameters['W'+str(l)] = np.random.randn(layer_dims[l],layer_dims[l-1])
        parameters['b'+str(l)] = np.zeros((layer_dims[l],1))
        
        assert(parameters['W' +str(l)].shape == (layer_dims[l],layer_dims[l-1]))
        assert(parameters['b'+str(l)].shape == (layer_dims[l],1))
        
    return parameters

In [19]:
parameters = initialize_parameters_deep([5,4,3])

for key in parameters:
    print(key + ": ",parameters[key])
print("W1 = " + str(parameters["W1"]))
print("b1 = " + str(parameters["b1"]))
print("W2 = " + str(parameters["W2"]))
print("b2 = " + str(parameters["b2"]))

W1:  [[ 1.78862847  0.43650985  0.09649747 -1.8634927  -0.2773882 ]
 [-0.35475898 -0.08274148 -0.62700068 -0.04381817 -0.47721803]
 [-1.31386475  0.88462238  0.88131804  1.70957306  0.05003364]
 [-0.40467741 -0.54535995 -1.54647732  0.98236743 -1.10106763]]
b1:  [[0.]
 [0.]
 [0.]
 [0.]]
W2:  [[-1.18504653 -0.2056499   1.48614836  0.23671627]
 [-1.02378514 -0.7129932   0.62524497 -0.16051336]
 [-0.76883635 -0.23003072  0.74505627  1.97611078]]
b2:  [[0.]
 [0.]
 [0.]]
W1 = [[ 1.78862847  0.43650985  0.09649747 -1.8634927  -0.2773882 ]
 [-0.35475898 -0.08274148 -0.62700068 -0.04381817 -0.47721803]
 [-1.31386475  0.88462238  0.88131804  1.70957306  0.05003364]
 [-0.40467741 -0.54535995 -1.54647732  0.98236743 -1.10106763]]
b1 = [[0.]
 [0.]
 [0.]
 [0.]]
W2 = [[-1.18504653 -0.2056499   1.48614836  0.23671627]
 [-1.02378514 -0.7129932   0.62524497 -0.16051336]
 [-0.76883635 -0.23003072  0.74505627  1.97611078]]
b2 = [[0.]
 [0.]
 [0.]]


# Step 3: Forward Propagation
Computing the linear part of the forward propagation

$$Z^{[l]} = W^{[l]}A^{[l-1]} +b^{[l]}$$
where $A^{[0]} = X$

In [20]:
def linear_forward(A,W,b):
    
    Z = np.dot(W,A) + b
    
    cache = (A,W,b)
    
    return Z, cache

In [22]:
def linear_activation_forward(A_prev, W, b, activation):
    if activation == "sigmoid":
        Z, linear_cache = linear_forward(A_prev, W, b)
        A, activation_cache = sigmoid(Z)
    elif activation == "relu":
        Z, linear_cache = linear_forward(A_prev, W, b)
        A, activation_cache = relu(Z)
        
    cache = (linear_cache, activation_cache)
    
    return A, cache

In [26]:
def L_model_forward(X, parameters):
    caches = []
    A=X
    L=len(parameters) // 2
    
    for l in range(1,L):
        A_prev = A
        
        A, cache = linear_activation_forward(A_prev, parameters["W"+str(l)], parameters["b"+str(l)], activation="relu")
        caches.append(cache)
        
    AL, cache = linear_activation_forward(A, parameters["W"+str(L)], parameters["b"+str(L)], activation="sigmoid")
    caches.append(cache)
    
    return AL, caches

# Step 4: Cost Function

Computing the cross-entropy cost $J$, using the following formula: $$-\frac{1}{m} \sum\limits_{i = 1}^{m} (y^{(i)}\log\left(a^{[L] (i)}\right) + (1-y^{(i)})\log\left(1- a^{[L](i)}\right)) \tag{7}$$

In [27]:
def compute_cost(AL, Y):
    
    m = Y.shape[1]
    
    cost = (-1/m) * np.sum(Y * np.log(AL) + (1-Y) * np.log(1-AL))
    
    cost = np.squeeze(cost)
    
    return cost

# Step 5: Backward Propagation

$$ dW^{[l]} = \frac{\partial \mathcal{J} }{\partial W^{[l]}} = \frac{1}{m} dZ^{[l]} A^{[l-1] T}$$
$$ db^{[l]} = \frac{\partial \mathcal{J} }{\partial b^{[l]}} = \frac{1}{m} \sum_{i = 1}^{m} dZ^{[l](i)}$$
$$ dA^{[l-1]} = \frac{\partial \mathcal{L} }{\partial A^{[l-1]}} = W^{[l] T} dZ^{[l]}$$

In [28]:
def linear_backward(dZ, cache):
    
    A_prev, W, b = cache
    m = A_prev.shape[1]
    
    dW = (1/m) * np.dot(dZ,A_prev.T)
    db = (1/m) * np.sum(dZ, axis = 1, keepdims=True)
    dA_prev = np.dot(W.T,dZ)
    
    return dA_prev, db, dW

In [29]:
def linear_activation_backward(dA, cache, activation):
    
    linear_cache, activation_cache = cache
    
    if activation == "relu":
        
        dZ = relu_backward(dA, activation_cache)
        dA_prev, dW, db = linear_backward(dZ, linear_cache)
        
    elif activation == "sigmoid":

        dZ = sigmoid_backward(dA, activation_cache)
        dA_prev, dW, db = linear_backward(dZ, linear_cache)
        
    return dA_prev, dW, db

In [30]:
def L_model_backward(AL, Y, caches):
    
    grads = {}
    L = len(caches) # the number of layers
    m = AL.shape[1]
    Y = Y.reshape(AL.shape) # after this line, Y is the same shape as AL

    dAL = - (np.divide(Y, AL) - np.divide(1 - Y, 1 - AL))

    current_cache = caches[L-1]
    dA_prev_temp, dW_temp, db_temp = linear_activation_backward(dAL, current_cache, activation="sigmoid")
    grads["dA" + str(L-1)] = dA_prev_temp
    grads["dW" + str(L)] = dW_temp
    grads["db" + str(L)] = db_temp

    for l in reversed(range(L-1)):
        current_cache = caches[l]
        dA_prev_temp, dW_temp, db_temp = linear_activation_backward(grads["dA" + str(l + 1)], current_cache, activation="relu")
        grads["dA" + str(l)] = dA_prev_temp
        grads["dW" + str(l + 1)] = dW_temp
        grads["db" + str(l + 1)] = db_temp
        
    return grads